In [ ]:
import pyttsx3
import speech_recognition as sr
import datetime
import pyaudio
import re
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
import warnings
import math
import requests
import mysql.connector
import random
import string
from twilio.rest import Client
warnings.filterwarnings("ignore", category=DeprecationWarning)


mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2003",
    database="hospital_appointments"
)

mycursor = mydb.cursor()

account_sid = 'account_sid Number'
auth_token = 'auth_token Number'
twilio_phone_number = '+Phone Number'



def text_to_number(text):
    number_words = {
        'zero' or 'Zero': 0, 'one' or 'One': 1, 'two' or 'Two': 2, 'three' or 'Three': 3, 'four' or 'Four': 4,
        'five' or 'Five': 5, 'six' or 'Six': 6, 'seven' or 'Seven': 7, 'eight' or 'Eight': 8, 'nine' or 'Nine': 9,
        'ten' or 'Ten': 10, 'eleven' or 'Eleven': 11, 'twelve' or 'Twelve': 12, 'thirteen' or 'Thirteen': 13,
        'fourteen' or 'Fourteen': 14, 'fifteen' or 'Fifteen': 15, 'sixteen' or 'Sixteen': 16,
        'seventeen' or 'Seventeen': 17,
        'eighteen' or 'Eighteen': 18, 'nineteen' or 'Nineteen': 19, 'twenty' or 'Twenty': 20

    }

    words = text.split()
    numerical_form = ''
    for word in words:
        numerical_form += str(number_words.get(word.lower(), '')) + ' '

    return numerical_form.strip()



def generate_appointment_id():
    characters = string.ascii_letters + string.digits
    appointment_id = ''.join(random.choice(characters) for _ in range(8))
    return appointment_id

def send_sms(phone_number, message):
    client = Client(account_sid, auth_token)

    if not phone_number.startswith('+91'):
        phone_number = '+91' + phone_number

    try:
        message = client.messages.create(
            body=message,
            from_=twilio_phone_number,
            to=phone_number
        )
        print(f"SMS sent to {phone_number}: {message.sid}")
    except Exception as e:
        print(f"Error sending SMS: {e}")


def get_hospital_number(hospital_name):
    mycursor.execute("SELECT hospital_number FROM hospitals WHERE hospital_name = %s", (hospital_name,))
    result = mycursor.fetchone()
    if result:
        return result[0]
    else:
        return None


def book_appointment():
    print("Enter your name:")
    speak("Enter your name:")
    name = recognize_speech()
    print("Enter your age: ")
    speak("Enter your age: ")
    num_age = recognize_speech().strip()
    age_num = num_age.split()[0]
    if age_num.isdigit():
        age = int(age_num)
    else:
        age = int(text_to_number(age_num))
    print("Enter the desired date and time")
    speak("Enter the desired date and time ")
    date_time = recognize_speech()
    print("Enter your mobile number:")
    speak("Enter your mobile number:")
    mobile_number = recognize_speech()
    print("Enter the hospital name:")
    speak("Enter the hospital name:")
    hospital_name = recognize_speech()
    hospital_number = get_hospital_number(hospital_name)


    if hospital_number is None:
        print("Hospital not found. Please enter a valid hospital name.")
        speak("Hospital not found. Please enter a valid hospital name.")
        return
    appointment_id = generate_appointment_id()


    sql = "INSERT INTO appointments (appointment_id, name, age, date_time, mobile_number, hospital_name) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (appointment_id, name, age, date_time, mobile_number, hospital_name)
    mycursor.execute(sql, values)
    mydb.commit()


    user_message = f"Your appointment at {hospital_name} on {date_time} has been booked successfully. Appointment ID: {appointment_id}"
    send_sms(mobile_number, user_message)

    hospital_message = f"New appointment booked: Name: {name}, Age: {age}, Date & Time: {date_time}, Mobile: {mobile_number}, Appointment ID: {appointment_id}"
    send_sms(hospital_number, hospital_message)

    print("Appointment booked successfully!")
    speak("Appointment booked successfully!.")


engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)
engine.setProperty('rate', 150)
engine.setProperty('volume', 2.0)


def recognize_speech(max_attempts=5):
    recognizer = sr.Recognizer()
    attempt = 1
    while attempt <= max_attempts:
        with sr.Microphone() as source:
            print("Listening...")
            recognizer.pause_threshold = 1
            recognizer.adjust_for_ambient_noise(source, duration=1)
            audio = recognizer.listen(source)
        try:
            print("Processing...")
            speak("Processing...")
            query = recognizer.recognize_google(audio, language='en-in')
            print(f"You said: {query}")
            return query.lower()
        except Exception as ex:
            print(ex)
            print("Please say that again.")
            speak("Please say that again.")
            attempt += 1
    print("Max attempts reached. Unable to recognize speech.")
    return "none"


def speak(audio):
    engine.say(audio)
    engine.runAndWait()


def wish():
    hour = int(datetime.datetime.now().hour)
    if 0 <= hour < 12:
        print("Good Morning! ")
        speak("Good Morning! ")
    elif 12 <= hour < 17:
        print("Good Afternoon!")
        speak("Good Afternoon!")
    elif 17 <= hour < 20:
        print("Good Evening!")
        speak("Good Evening!")
    else:
        print("Good Night! ")
        speak("Good Night!")


def read_hospital_data():
    hospital_data = []
    with open('Original_Mediacal_Hospital_Data_1.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            hospital_data.append({
                'name': row[0],
                'address': row[1],
                'latitude': float(row[2]),
                'longitude': float(row[3]),
                'ratings': float(row[4])
            })
    return hospital_data


def calculate_distance(lat1, lon1, lat2, lon2):
    radius = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c
    return distance


def find_nearest_hospitals(user_lat, user_lon, hospital_data, num_hospitals=2):
    nearest_hospitals = []
    for hospital in hospital_data:
        distance = calculate_distance(user_lat, user_lon, hospital['latitude'], hospital['longitude'])
        nearest_hospitals.append((hospital, distance))

    nearest_hospitals.sort(key=lambda x: x[1])
    return nearest_hospitals[:num_hospitals]


def display_nearest_hospitals(nearest_hospitals):
    print("\nNearest Hospitals:")
    speak("Nearest Hospitals:")
    for hospital, distance in nearest_hospitals:
        print(f"\nName: {hospital['name']}")
        speak(f"\nName: {hospital['name']}")
        print(f"Address: {hospital['address']}")
        speak(f"Address: {hospital['address']}")
        print(f"Distance: {distance:.2f} km")
        speak(f"Distance: {distance:.2f} km")
        print(f"Ratings: {hospital['ratings']}")
        speak(f"Ratings: {hospital['ratings']}")
    print()


def get_lat_lng(location):
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location,
        'format': 'json',
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        lat = float(data[0]['lat'])
        lon = float(data[0]['lon'])
        return lat, lon
    else:
        print(f"Failed to find coordinates for {location}")
        speak(f"Failed to find coordinates for {location}")
        return None, None


def Location():
    try:
        print("Enter your current Location :  ")
        speak("Enter your current Location :  ")
        location = recognize_speech()
        user_lat, user_lon = get_lat_lng(location)

        if user_lat is not None and user_lon is not None:
            hospital_data = read_hospital_data()
            nearest_hospitals = find_nearest_hospitals(user_lat, user_lon, hospital_data)
            display_nearest_hospitals(nearest_hospitals)
        else:
            print("Unable to find latitude and longitude for the given location.")
            speak("Unable to find latitude and longitude for the given location.")
    except Exception as e:
        print("An error occurred:",e)
        speak("An error occurred:",e)


training = pd.read_csv('Training.csv')
testing= pd.read_csv('Testing.csv')
cols= training.columns
cols= cols[:-1]
x = training[cols]
y = training['prognosis']
y1= y


reduced_data = training.groupby(training['prognosis']).max()
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']
testy    = le.transform(testy)


clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
scores = cross_val_score(clf, x_test, y_test, cv=3)
model=SVC()
model.fit(x_train,y_train)


importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols


def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()


severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()


symptoms_dict = {}
for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index

def calc_condition(exp,days):
    sum=0
    for item in exp:
         sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp)+1)>13):
        print("You should take the consultation from doctor. ")
        speak("You should take the consultation from doctor. ")
    else:
        print("It might not be that bad but you should take precautions.")
        speak("It might not be that bad but you should take precautions.")



def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)


def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)


def getInfo():
    print("Enter Your Name  : ")
    speak("Enter Your Name  : ")
    name=recognize_speech()
    print("Hello, ",name)
    speak("Hello, "+name)


def check_pattern(dis_list,inp):
    pred_list=[]
    inp=inp.replace(' ','_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list=[item for item in dis_list if regexp.search(item)]
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return 0,[]


def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
      input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])


def print_disease(node):
    node = node[0]
    val  = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x:x.strip(),list(disease)))


def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    chk_dis=",".join(feature_names).split(",")
    symptoms_present = []

    while True:

        print("\nEnter the symptom you are experiencing  \t\t")
        speak("Enter the symptom Are you Experiencing ?")
        dis_input = recognize_speech().strip()
        disease_input = dis_input.split()[2]
        conf,cnf_dis=check_pattern(chk_dis,disease_input)
        if conf==1:
            print("searches related to input: ")
            speak("searches related to input: ")
            for num,it in enumerate(cnf_dis):
                print(num,")",it)
                speak(f"{num} ,) {it}")
            if num!=0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                speak(f"Select the one you meant (0 - {num}):  ")

                conf_num = recognize_speech().strip()
                input_num = conf_num.split()[1]
                if input_num.isdigit():
                    conf_inp = int(input_num)
                else:
                    conf_inp = text_to_number(input_num)
            else:
                conf_inp = 0

            disease_input = cnf_dis[int(conf_inp)]
            break

        else:
            print("Enter valid symptom.")
            speak("Enter valid symptom.")

    while True:
        try:
            print("Okay. how many days you have This Probleam ? : ")
            speak("Okay. how many days you have This Probleam ? : ")
            num_days_input = recognize_speech().strip()
            # Accepting input as string and stripping any leading/trailing spaces
            days = num_days_input.split()[0]
            if days.isdigit():
                num_days = int(days)
            else:
                num_days = text_to_number(days)

            break
        except(ValueError, IndexError):
            print("Enter valid input.")
            speak("Enter valid input.")


    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])

            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]


            symptoms_exp=[]
            for syms in list(symptoms_given):
                inp=""
                print("Are you experiencing " + syms, "? : ", end='')
                speak(f"Are you experiencing {syms}?")

                while True:
                    inp=recognize_speech()
                    if(inp=="yes" or "yes alright" or inp=="no"):
                        break
                    else:
                        print("provide proper answers i.e. (yes Alright /no) : ",end="")
                        speak("provide proper answers i.e. (yes alright /no) : ")
                if(inp=="yes" or "yes alright"):
                    symptoms_exp.append(syms)

            second_prediction=sec_predict(symptoms_exp)
            calc_condition(symptoms_exp,int(num_days))
            if(present_disease[0]==second_prediction[0]):
                print("You may have ", present_disease[0])
                speak(f"You may have {present_disease[0]}")
                print(description_list[present_disease[0]])
                speak(description_list[present_disease[0]])


            else:
                print("You may have ", present_disease[0], "or ", second_prediction[0])
                speak(f"You may have {present_disease[0]} or {second_prediction[0]}")
                print(description_list[present_disease[0]])
                speak(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])
                speak(description_list[second_prediction[0]])


            precution_list=precautionDictionary[present_disease[0]]
            print("Take following measures : ")
            speak("Take following measures : ")
            for  i,j in enumerate(precution_list):
                print(i+1,")",j)
                speak(f"{i + 1}, {j}")
    recurse(0, 1)
if __name__ == "__main__":
    wish()
    getSeverityDict()
    getDescription()
    getprecautionDict()
    getInfo()
    tree_to_code(clf,cols)
    print("----------------------------------------------------------------------------------------")
    Location()
    book_appointment()

mydb.close()

